<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/data2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 13.5 MB/s eta 0:00:00


In [17]:
pip install resampy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.7 MB/s eta 0:00:00


In [2]:
!pip install transformers



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.6 MB/s eta 0:00:00


In [18]:
import torch
import torch.nn as nn
import numpy as np
import cv2
import urllib
from transformers import AutoImageProcessor, Data2VecVisionModel, AutoProcessor, Data2VecAudioModel
from PIL import Image
from moviepy.editor import VideoFileClip
from datasets import load_dataset
import librosa
from resampy import resample

## Frame Embedding

In [6]:
video_path = '/content/The Present - OFFICIAL.mp4'
cap = cv2.VideoCapture(video_path)
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # resize the frame to (640, 480)
    resized_frame = cv2.resize(frame, (640, 480))
    rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    frames.append(resized_frame)
    del frame, resized_frame, rgb_frame
cap.release()

pil_frames = []
for frame in frames:
    pil_frame = Image.fromarray(frame)
    pil_frames.append(pil_frame)
    del pil_frame

In [10]:
image_processor = AutoImageProcessor.from_pretrained("facebook/data2vec-vision-base")
model = Data2VecVisionModel.from_pretrained("facebook/data2vec-vision-base")

frame_embedding = []

for image in pil_frames:
  inputs = image_processor(image, return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs)

  last_hidden_states = outputs.last_hidden_state
  frame_embedding.append(last_hidden_states)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.9/dist-packages/transformers/models/beit/image_processing_beit.py:108: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


KeyboardInterrupt: ignored

## Sound Embedding

In [12]:
# load the video clip
video_path = '/content/The Present - OFFICIAL.mp4'
clip = VideoFileClip(video_path)

# extract the audio from the video
audio = clip.audio

# write the audio to a file
audio_path = '/content/audio.wav'
audio.write_audiofile(audio_path)

MoviePy - Writing audio in /content/audio.wav


MoviePy - Done.


In [23]:
audio_input, sample_rate = librosa.load("/content/audio.wav", mono = True, sr = None)
data = resample(audio_input, sample_rate, 16000)


In [25]:
processor = AutoProcessor.from_pretrained("facebook/data2vec-audio-base-960h")
model = Data2VecAudioModel.from_pretrained("facebook/data2vec-audio-base-960h")

inputs = processor(data, sampling_rate=16000, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: ignored